In [2]:
def writeData(mode, id, data):
    if mode == 'addSentEvents':
        if not data.empty:
            data.loc[:, 'eventnumber'] = ''
        prevSentEvents = readData(mode='sentEvents', id=id)
        if not prevSentEvents.empty:
            today = pd.to_datetime(date.today())
            eventTimes = pd.to_datetime(prevSentEvents.loc[:, 'eventtime'])
            oldEvents = prevSentEvents[eventTimes < today].index
            prevSentEvents.drop(labels=oldEvents, inplace=True)
            print(f'{len(oldEvents)} events was deleted. ', end='')
        # print(f'type oldevents: {type(data)}, type prevSentEvents: {type(prevSentEvents)}')
        sentEvents = prevSentEvents.append(data)  # copy=false?
        sentEvents.reset_index(drop=True, inplace=True)
        dataFilename = 'data/' + id + '_ggb_sentEvents.csv'
        fileHandle = open(dataFilename, 'w', encoding="utf-8")
        sentEvents.to_csv(path_or_buf=fileHandle, sep='\t', line_terminator='\n')
    elif mode == 'lastEvents':
        dataFilename = 'data/' + id + '_ggb_lastEvents.csv'
        fileHandle = open(dataFilename, 'w', encoding="utf-8")
        data.to_csv(path_or_buf=fileHandle, sep='\t', line_terminator='\n')
    elif mode == 'settings':
        dataFilename = 'data/' + id + '_ggb_settings.csv'
        fileHandle = open(dataFilename, 'w', encoding="utf-8")
        fileHandle.write(data)
    else:
        print(f'{mode} is wrong mode for writing')
    fileHandle.close()
    print(f'{mode} is written to: ', dataFilename)


def readData(mode, id):
    if mode in ['lastEvents', 'sentEvents']:
        try:
            if mode == 'lastEvents':
                dataFilename = 'data/' + id + '_ggb_lastEvents.csv'
            elif mode == 'sentEvents':
                dataFilename = 'data/' + id + '_ggb_sentEvents.csv'
            fileHandle = open(dataFilename, 'r', encoding="utf-8")
            data = pd.read_csv(filepath_or_buffer=fileHandle, sep='\t', lineterminator='\n', index_col=0)
            fileHandle.close()
            print(f'{mode} data was read from: {dataFilename}')
        except FileNotFoundError:
            data = pd.DataFrame([], columns=['eventnumber', 'eventid', 'eventartist',
                                             'eventtime', 'eventvenue', 'eventcity', 'eventcountry'])
            print(f'{mode} data not found. Empty DF created')
    elif mode == 'settings':
        dataFilename = 'data/' + id + '_ggb_settings.csv'
        fileHandle = open(dataFilename, 'r', encoding="utf-8")
        settings = fileHandle.read()
        data = dict() if settings == '' else eval(settings)
        fileHandle.close()
        print(f'{mode} data was read from: {dataFilename}')
    else:
        print(f'{mode} is wrong mode for writing')
    return data


def readSett(listOf, id):
    data = []
    settList = {'lastfmUser', 'timeDelay', 'minListens', 'places'}
    if len(set(listOf).intersection(settList)):
        settings = readData('settings', id)
    for sett in listOf:
        if sett in settList:
            data.append(settings[sett])
        elif sett in ('sentEvents', 'lastEvents'):
            data.append(readData(sett, id))
    return tuple(data)


def writeSett(id, sett, value):
    settings = readData('settings', id)
    settings[sett] = value
    writeData(mode='settings', id=id, data=str(settings))

In [37]:
def addCountry(countryAnswer, userId):
    fileHandle = open('data/cities/worldcities/WorldCountryList.csv', 'r', encoding="utf-8")
    data = pd.read_csv(filepath_or_buffer=fileHandle, sep='\t', lineterminator='\n',index_col=0)
    try:
        mask = np.column_stack([data[col] == countryAnswer.strip().lower() for col in data])
        addedCountry = data.loc[mask].iat[0, 0]
        places = readSett(['places'], userId)[0]
        if places == ():
            places = dict()
        places[addedCountry] = []
        writeSett(userId, 'places', places)
    except IndexError:
        addedCountry = False
    fileHandle.close()
    return addedCountry

def addCity(cityAnswer, country, userId):
    fileHandle = open('data/RussiaCityList.csv', 'r', encoding="utf-8",)
    data = pd.read_csv(filepath_or_buffer=fileHandle, sep='\t', lineterminator='\n', index_col=0)
    try:
        mask = np.column_stack([data[col] == cityAnswer.strip().lower() for col in data])
        addedCity = data.loc[mask].iat[0, 0]
        places = readSett(['places'], userId)[0]
        if addedCity in places[country]:
            return addedCity
        places[country].append(addedCity)
        writeSett(userId, 'places', places)
    except IndexError:
        addedCity = False
    fileHandle.close()
    return addedCity

In [38]:
import pandas as pd
import numpy as np
userId = '144297913'
countryAnswer = 'russia'

In [39]:
addC = addCountry(countryAnswer, userId)

In [40]:
addC

False

In [42]:
fileHandle = open('data/cities/worldcities/WorldCountryList.csv', 'r', encoding="utf-8")
data = pd.read_csv(filepath_or_buffer=fileHandle, sep='\t', lineterminator='\n',index_col=0)
data

,0
0,"['Afghanistan', 'Albania', 'Algeria', 'America..."


In [35]:
mask = np.column_stack([data[col].str.lower() == countryAnswer.strip().lower() for col in data])
addedCountry = data.loc[mask].iat[0, 0]
addedCountry

,country_orig,country0,country1,country2,country3,country4
169,Russia,russia,russian federation,россия,российская федерация,рф
